In [1]:
import boto3
import polars as pl
import botocore
import tarfile
import os
from io import BytesIO
from pypdf import PdfReader, PdfWriter
import gzip
from io import BytesIO
from fastwarc import ArchiveIterator
from fastwarc.stream_io import GZipStream
from resiliparse.extract.html2text import extract_plain_text
from resiliparse.parse.lang import detect_fast
from resiliparse.parse.html import HTMLTree
from fastwarc.warc import is_http
from surt import surt
import tldextract
import idna
import re
from urllib.parse import urljoin, urlparse
from resiliparse.parse.encoding import detect_encoding, bytes_to_str
import boto3
import polars as pl
from fastwarc.warc import ArchiveIterator, WarcRecordType
from fastwarc.stream_io import GZipStream
import gcsfs # Ensure you have this installed for GCS filesystem handling
import os

client = boto3.client('s3', 
                      aws_access_key_id=os.getenv('ASCII_AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('ASCII_AWS_SECRET_ACCESS_KEY'))
#client = boto3.client('s3', 
#        region_name='us-east-1',  # Common Crawl is in us-east-1
#        config=Config(signature_version=UNSIGNED)
#    )

In [2]:
crawls = [
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251115220136-05256.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251115234907-05257.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116014712-05258.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116034034-05259.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116054722-05260.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116072531-05261.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116085944-05262.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116103419-05263.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116120640-05264.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116134805-05265.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116152516-05266.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116170848-05267.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116190217-05268.warc.gz']
key = 'crawl-data/CC-NEWS/index.html'
response = client.get_object(Bucket='commoncrawl', Key=key)

In [3]:
response['Body'].read()

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <title>Common Crawl News Dataset</title>\n  <meta charset="utf-8">\n  <link rel="stylesheet" href="https://data.commoncrawl.org/static/bucket.css">\n  <style>\n    th,td { padding: 0.2em 0.5em; }\n    th { background-color: #e0e0e0; }\n    tr { background-color: #f0f0f0; }\n    tr:nth-child(odd) { background-color: #fcfcfc; }\n    footer { width: 100%; margin-top: 1em; }\n  </style>\n</head>\n\n<body>\n\n<h1>Common Crawl News Dataset</h1>\n\n<p>The news dataset includes articles from news sites all over the world. WARC files are released on a daily basis. The news crawl was started in 2016, please see the <a href="https://commoncrawl.org/2016/10/news-dataset-available/">news dataset announcement</a> for further information.</p>\n\n<p>The <a href="https://github.com/commoncrawl/news-crawl/">source code of the news crawler</a> is available on our <a href="https://github.com/commoncrawl/">GitHub account</a>.\n\nPlease, <a href="https://github

In [2]:
key = 'crawl-data/CC-NEWS/2025/index.html'
response = client.get_object(Bucket='commoncrawl', Key=key)

In [7]:
key_gz = 'crawl-data/CC-NEWS/2025/11/warc.paths.gz'

response = client.get_object(Bucket='commoncrawl', Key=key_gz)
    
gzipped_body_bytes = response['Body'].read()
    
decompressed_bytes = gzip.decompress(gzipped_body_bytes)


In [8]:
current_data = pl.read_csv(BytesIO(decompressed_bytes), has_header=False)
print(current_data.shape)
current_data.head()

(574, 1)


column_1
str
"""crawl-data/CC-NEWS/2025/11/CC-…"
"""crawl-data/CC-NEWS/2025/11/CC-…"
"""crawl-data/CC-NEWS/2025/11/CC-…"
"""crawl-data/CC-NEWS/2025/11/CC-…"
"""crawl-data/CC-NEWS/2025/11/CC-…"


In [2]:
key = 'crawl-data/CC-NEWS/index.html'
response = client.get_object(Bucket='commoncrawl', Key=key)
print(response['Body'].read())

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <title>Common Crawl News Dataset</title>\n  <meta charset="utf-8">\n  <link rel="stylesheet" href="https://data.commoncrawl.org/static/bucket.css">\n  <style>\n    th,td { padding: 0.2em 0.5em; }\n    th { background-color: #e0e0e0; }\n    tr { background-color: #f0f0f0; }\n    tr:nth-child(odd) { background-color: #fcfcfc; }\n    footer { width: 100%; margin-top: 1em; }\n  </style>\n</head>\n\n<body>\n\n<h1>Common Crawl News Dataset</h1>\n\n<p>The news dataset includes articles from news sites all over the world. WARC files are released on a daily basis. The news crawl was started in 2016, please see the <a href="https://commoncrawl.org/2016/10/news-dataset-available/">news dataset announcement</a> for further information.</p>\n\n<p>The <a href="https://github.com/commoncrawl/news-crawl/">source code of the news crawler</a> is available on our <a href="https://github.com/commoncrawl/">GitHub account</a>.\n\nPlease, <a href="https://github

In [3]:
from bs4 import BeautifulSoup
import pandas as pd
soup = BeautifulSoup(response['Body'].read(), "html.parser")


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [4]:
key = 'crawl-data/CC-NEWS/index.html'
output_path = 'gs://gen-ai-tu/news/index/'
res0 = client.get_object(Bucket='commoncrawl', Key=key)
years = pd.read_html(res0['Body'].read())
index = []
for i1, r1 in years[0].iterrows():
    monthly_key = f'crawl-data/CC-NEWS/{int(r1['Year'])}/index.html'
    res1 = client.get_object(Bucket='commoncrawl', Key=monthly_key)
    month = pd.read_html(res1['Body'].read())
    for i2, r2 in month[0].iterrows():
        monty_key = f'crawl-data/CC-NEWS/{int(r1['Year'])}/{r2['WARC file list']}'
        res2 = client.get_object(Bucket='commoncrawl', Key=monty_key)
        decompressed_bytes = gzip.decompress(res2['Body'].read())
        data = pl.read_csv(BytesIO(decompressed_bytes), has_header=False)
        index.append(data.with_columns(
            pl.lit(int(r1['Year'])).alias('year'),
            pl.lit(int(r2['Month'])).alias('month'),
            pl.col('column_1').str.split('-').list.get(4).str.slice(6,2).cast(pl.Int8).alias('day')
            ).rename({'column_1': 'warc_destination'})
        )
index_df = pl.concat(index)
#index_df.write_parquet(
#                output_path,
#                partition_by=['year', 'month', 'day'],
#                #use_pyarrow=True ,
#                storage_options={
#                    "project_id": os.getenv('GCP_PROJECT'),
#                    "private_key_id": os.getenv('GCP_PRIVATE_KEY_ID'),
#                    "private_key": os.getenv('GCP_PRIVATE_KEY'),
#                    "client_email": os.getenv('GCP_EMAIL')},
#                mkdir=True,
#            )

In [13]:
monthly_key = f'crawl-data/CC-NEWS/2026/index.html'
res1 = client.get_object(Bucket='commoncrawl', Key=monthly_key)
month = pd.read_html(res1['Body'].read())

In [14]:
month

[Empty DataFrame
 Columns: [Month, Num. WARC files, Total WARC Size Compressed (TiB), WARC file list]
 Index: []]

In [5]:
index_df.head()

warc_destination,year,month,day
str,i32,i32,i8
"""crawl-data/CC-NEWS/2025/01/CC-…",2025,1,1
"""crawl-data/CC-NEWS/2025/01/CC-…",2025,1,1
"""crawl-data/CC-NEWS/2025/01/CC-…",2025,1,1
"""crawl-data/CC-NEWS/2025/01/CC-…",2025,1,1
"""crawl-data/CC-NEWS/2025/01/CC-…",2025,1,1


In [ ]:
key = 'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101004549-04954.warc.gz'


2025
2024
2023
2022
2021
2020
2019
2018
2017
2016


In [ ]:
import boto3
import re
import polars as pl
from botocore import UNSIGNED
from botocore.config import Config
from botocore.exceptions import ClientError
from datetime import datetime

# 1. Configure S3 for anonymous access in US-EAST-1
# Specifying the region is crucial for public buckets to avoid redirect errors
s3_client = boto3.client('s3', region_name='us-east-1', config=Config(signature_version=UNSIGNED))
BUCKET = 'commoncrawl'

# CC-NEWS roughly started mid-2016
start_year = 2016
current_date = datetime.now()
end_year = current_date.year

# Regex to find WARC files inside the HTML
# Looks for: href="CC-NEWS-20240101..."
warc_pattern = re.compile(r'href="(CC-NEWS-.*?\.warc\.gz)"')

all_records = []

print(f"Scanning CC-NEWS indices from {start_year} to {end_year}...")

for year in range(start_year, end_year + 1):
    end_month = 12 if year < end_year else current_date.month
    
    for month in range(1, end_month + 1):
        month_str = f"{month:02d}"
        
        # We access the index.html explicitly
        # This file exists to allow browsers to view the files, and we can scrape it.
        prefix = f"crawl-data/CC-NEWS/{year}/{month_str}/"
        key_html = f"{prefix}index.html"
        
        try:
            print(f"Scanning: {key_html}...", end='\r')
            
            # Download the HTML content
            response = s3_client.get_object(Bucket=BUCKET, Key=key_html)
            html_content = response['Body'].read().decode('utf-8')
            
            # Extract all WARC filenames from this month's index
            warc_files = warc_pattern.findall(html_content)
            
            # Create a record for each file found
            for filename in warc_files:
                # We extract the date from the filename itself for accuracy
                # Filename: CC-NEWS-20251101004549-04954.warc.gz
                # Slicing:  [8:12]=Year, [12:14]=Month, [14:16]=Day
                file_year = filename[8:12]
                file_month = filename[12:14]
                file_day = filename[14:16]
                
                full_address = f"{prefix}{filename}"
                
                all_records.append({
                    "year": file_year,
                    "month": file_month,
                    "day": file_day,
                    "address": full_address
                })
                
        except ClientError as e:
            # 404/403 means the month is empty or didn't exist (common in 2016)
            pass
        except Exception as e:
            print(f"\nError processing {key_html}: {e}")

print("\n\nProcessing complete.")

if all_records:
    # Create Polars DataFrame
    df = pl.DataFrame(all_records)
    
    # Sort just to be neat
    df = df.sort(["year", "month", "day", "address"])
    
    print(f"Total WARC files found: {len(df)}")
    print(df.head())
    
    # df.write_csv("cc_news_full_list.csv")
else:
    print("No WARC files found. Check your network connection.")

Scanning CC-NEWS indices from 2016 to 2025...
Scanning: crawl-data/CC-NEWS/2025/12/index.html...

Processing complete.
No WARC files found. Check your network connection.


In [8]:
ip_pattern = re.compile(r"^(?:www\.)?\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\Z")
host_part_pattern = re.compile(
    r"^[a-z0-9]([a-z0-9_-]{0,61}[a-z0-9])?\Z", re.IGNORECASE | re.ASCII
)


In [9]:
from google.oauth2 import service_account

key_path = "../../../gcp_service_account.json"  
credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],)


In [ ]:
def get_surt_host(url):  # noqa: C901
    extracted = tldextract.extract(url, include_psl_private_domains=True)
    registered_domain = extracted.top_domain_under_public_suffix

    if registered_domain == "":
        registered_domain = f"{extracted.subdomain}.{extracted.domain}"
        if registered_domain == "":
            try:
                # Fallback to urlparse if tldextract fails
                host = urlparse(url).hostname
            except Exception as e:
                print(f"Failed to parse URL {url}: {e}")
                return None
            if not host:
                return None
        else:
            host = registered_domain
    else:
        host = registered_domain

    host = host.strip().lower()
    if len(host) < 1 or len(host) > 253:
        return None
    if ip_pattern.match(host):
        return None
    parts = host.split(".")
    if parts[-1] == "":
        # trailing dot is allowed, strip it
        parts = parts[0:-1]
    if len(parts) <= 1:
        # do not accept single-word hosts, must be at least `domain.tld'
        return None
    if len(parts) > 2 and parts[0] == "www":
        # strip leading 'www' to reduce number of "duplicate" hosts,
        # but leave at least 2 trailing parts (www.com is a valid domain)
        parts = parts[1:]
    for i, part in enumerate(parts):
        if len(part) > 63:
            return None
        if not host_part_pattern.match(part):
            try:
                idn = idna.encode(part).decode("ascii")
            except (
                idna.IDNAError,
                idna.core.InvalidCodepoint,
                UnicodeError,
                IndexError,
                Exception,
            ):
                print("Invalid host name: {}".format(url))
                return None

            # TODO: idna verifies the resulting string for length restrictions or invalid chars,
            #       maybe no further verification is required:
            if host_part_pattern.match(idn):
                parts[i] = idn
            else:
                print("Invalid host name: {}".format(url))
                return None
    parts.reverse()
    return ".".join(parts)

In [11]:
fs = gcsfs.GCSFileSystem()

In [12]:
KEY_PATH = "../../../gcp_service_account.json"
GCS_BUCKET_ROOT = "gen-ai-tu/news/raw"
LOCAL_TEMP_DIR = "./temp_staging"

# Initialize GCS FileSystem (Pure Python, more robust than C++ writer)
fs = gcsfs.GCSFileSystem(token=KEY_PATH)

# Clean start: ensure temp dir exists and is empty
if os.path.exists(LOCAL_TEMP_DIR):
    shutil.rmtree(LOCAL_TEMP_DIR)
os.makedirs(LOCAL_TEMP_DIR)

In [ ]:
output_path = 'gs://gen-ai-tu/news/raw'
for key in current_data['column_1'].to_list():
    print(f"Processing {key}...")
    try:
        # --- FIX 1: Initialize stream INSIDE the loop ---
        s3_response = client.get_object(Bucket='commoncrawl', Key=key)
        s3_stream = s3_response['Body']
        stream = GZipStream(s3_stream)
        
        tmp = []
        for record in ArchiveIterator(stream, record_types=WarcRecordType.response, func_filter=is_http):
            try:
                uri = record.headers.get('WARC-Target-URI')
                body_bytes = record.reader.read()
                html = bytes_to_str(body_bytes, detect_encoding(body_bytes))
                text = extract_plain_text(html)
                http_date =record.http_date 
                http_last_modified = record.http_last_modified 
                http_charset= record.http_charset 
                surt_uri = surt(uri)
                host = get_surt_host(uri)
                r = detect_fast(text, n_results=3)
                langs = []
                confs = []
                for i in range(len(r)):
                    langs.append(r[i][0])
                    confs.append(r[i][1])
                tmp.append({
                    'uri': uri,
                    'tree': html,
                    'text': text,
                    'main_lang': r[0][0], 
                    'langs': langs, 
                    'confs': confs,
                    'http_date': http_date,
                    'http_last_modified': http_last_modified,
                    'http_charset': http_charset,
                    'surt_uri': surt_uri,
                    'host': host})
            except Exception as e:
                print(f"Error processing record: {e}")
        if not tmp:
            print(f"No valid records found in {key}, skipping write.")
            continue
        try:
            path_parts = key.split('/')
            filename = path_parts[-1]
            year_str = path_parts[-3]  # '2025'
            month_str = path_parts[-2] # '11'
            timestamp = filename.split('-')[2] # '20251116190217'
            day_str = timestamp[6:8]
            day_str

            df = pl.from_dicts(tmp).with_columns(
                pl.lit(filename).alias('path'),
                pl.lit(year_str).alias('year'),
                pl.lit(month_str).alias('month'),
                pl.lit(day_str).alias('day')
            )
local_write_path = f"{LOCAL_TEMP_DIR}/{filename}" # Use subfolder to avoid collisions
        
        df.write_parquet(
            local_write_path,
            partition_by=['year', 'month', 'day', 'path', 'main_lang'],
            use_pyarrow=True 
        )

        # 5. Upload to GCS using Python GCSFS
        # recursive=True ensures the hive partition folders are uploaded correctly
        # We copy from local_write_path to the bucket root
        print(f"Uploading {filename} to GCS...")
        
        # fs.put will copy the *contents* of the local folder to the remote path
        fs.put(local_write_path, f"gs://{GCS_BUCKET_ROOT}", recursive=True)

        # 6. Cleanup Local Disk
        shutil.rmtree(local_write_path)
        print(f"Done with {key}")
        except Exception as write_err:
            print(f"Failed to write parquet to GCS for {key}: {write_err}")

    except Exception as e:
        print(f"Error processing key {key}: {e}")

Processing crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101004549-04954.warc.gz...
Failed to write parquet to GCS for crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101004549-04954.warc.gz: google::cloud::Status(UNAVAILABLE: Retry policy exhausted, with a last message of Could not create a OAuth2 access token to authenticate the request. The request was not sent, as such an access token is required to complete the request successfully. Learn more about Google Cloud authentication at https://cloud.google.com/docs/authentication. The underlying error message was: PerformWork() - CURL error [6]=Could not resolve hostname error_info={reason=, domain=, metadata={gcloud-cpp.retry.on-entry=false, gcloud-cpp.retry.function=CreateResumableUpload, gcloud-cpp.retry.reason=retry-policy-exhausted, gcloud-cpp.retry.original-message=Could not create a OAuth2 access token to authenticate the request. The request was not sent, as such an access token is required to complete the request successfully. Learn more abo

KeyboardInterrupt: 

In [29]:
key = 'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101004549-04954.warc.gz'
path_parts = key.split('/')
filename = path_parts[-1]
year_str = path_parts[-3]  # '2025'
month_str = path_parts[-2] # '11'
        
        # Extract Day from filename timestamp
timestamp = filename.split('-')[2] # '20251116190217'
day_str = timestamp[6:8]
day_str

'01'

In [33]:
import pyarrow as pa

In [34]:
df.with_columns(pl.lit(day_str).alias('day')).write_parquet(
                output_path, 
                partition_by=['year','month', 'path', 'main_lang'],
                use_pyarrow=True, # Often more stable for remote writes
                storage_options={"token": key_path}
            )

ModuleNotFoundError: pa.Table requires 'pyarrow' module to be installed

In [14]:
import shutil

In [ ]:
local_write_path = f"{LOCAL_TEMP_DIR}/{filename}" # Use subfolder to avoid collisions
df.write_parquet(
            local_write_path,
            partition_by=['year', 'month', 'day', 'path', 'main_lang'],
            use_pyarrow=True 
        )

        # 5. Upload to GCS using Python GCSFS
        # recursive=True ensures the hive partition folders are uploaded correctly
        # We copy from local_write_path to the bucket root
print(f"Uploading {filename} to GCS...")
        
        # fs.put will copy the *contents* of the local folder to the remote path
fs.put(local_write_path, f"gs://{GCS_BUCKET_ROOT}", recursive=True)



Uploading CC-NEWS-20251101004549-04954.warc.gz to GCS...


NameError: name 'shutil' is not defined

In [15]:
        # 6. Cleanup Local Disk
shutil.rmtree(local_write_path)
print(f"Done with {key}")

NotADirectoryError: [Errno 20] Not a directory: './temp_staging/CC-NEWS-20251101004549-04954.warc.gz'

In [ ]:
import os
import gzip
import re
import boto3
from io import BytesIO
from urllib.parse import urlparse
import tldextract
import idna
from datetime import datetime

# Spark Imports
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType, LongType
from pyspark.sql import Row

# Processing Imports (Must be installed on all worker nodes)
from fastwarc.warc import ArchiveIterator, WarcRecordType, is_http
from fastwarc.stream_io import GZipStream
from resiliparse.extract.html2text import extract_plain_text
from resiliparse.parse.lang import detect_fast
from resiliparse.parse.encoding import detect_encoding, bytes_to_str
from surt import surt

# --- Configuration ---
AWS_ACCESS_KEY = os.getenv('ASCII_AWS_ACCESS_KEY_ID')
AWS_SECRET_KEY = os.getenv('ASCII_AWS_SECRET_ACCESS_KEY')
GCP_SA_KEY_PATH = "../../../gcp_service_account.json"
GCS_BUCKET_ROOT = "gs://gen-ai-tu/news/raw"

# --- 1. Helper Functions (Must be available to workers) ---
ip_pattern = re.compile(r"^(?:www\.)?\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\Z")
host_part_pattern = re.compile(r"^[a-z0-9]([a-z0-9_-]{0,61}[a-z0-9])?\Z", re.IGNORECASE | re.ASCII)

def get_surt_host(url):  # noqa: C901
    extracted = tldextract.extract(url, include_psl_private_domains=True)
    registered_domain = extracted.top_domain_under_public_suffix

    if registered_domain == "":
        registered_domain = f"{extracted.subdomain}.{extracted.domain}"
        if registered_domain == "":
            try:
                # Fallback to urlparse if tldextract fails
                host = urlparse(url).hostname
            except Exception as e:
                print(f"Failed to parse URL {url}: {e}")
                return None
            if not host:
                return None
        else:
            host = registered_domain
    else:
        host = registered_domain

    host = host.strip().lower()
    if len(host) < 1 or len(host) > 253:
        return None
    if ip_pattern.match(host):
        return None
    parts = host.split(".")
    if parts[-1] == "":
        # trailing dot is allowed, strip it
        parts = parts[0:-1]
    if len(parts) <= 1:
        # do not accept single-word hosts, must be at least `domain.tld'
        return None
    if len(parts) > 2 and parts[0] == "www":
        # strip leading 'www' to reduce number of "duplicate" hosts,
        # but leave at least 2 trailing parts (www.com is a valid domain)
        parts = parts[1:]
    for i, part in enumerate(parts):
        if len(part) > 63:
            return None
        if not host_part_pattern.match(part):
            try:
                idn = idna.encode(part).decode("ascii")
            except (
                idna.IDNAError,
                idna.core.InvalidCodepoint,
                UnicodeError,
                IndexError,
                Exception,
            ):
                print("Invalid host name: {}".format(url))
                return None

            # TODO: idna verifies the resulting string for length restrictions or invalid chars,
            #       maybe no further verification is required:
            if host_part_pattern.match(idn):
                parts[i] = idn
            else:
                print("Invalid host name: {}".format(url))
                return None
    parts.reverse()
    return ".".join(parts)

In [9]:
def process_warc_partition(iterator):
    """
    This runs on the worker nodes. 
    It receives an iterator of rows (each row contains a 'warc_path').
    It initializes its own boto3 client and processes files.
    """
    s3_client = boto3.client('s3', 
                             aws_access_key_id=AWS_ACCESS_KEY,
                             aws_secret_access_key=AWS_SECRET_KEY)
    
    for row in iterator:
        key = row.warc_path
        
        # Extract filename metadata for partitioning
        try:
            path_parts = key.split('/')
            filename = path_parts[-1]
            year_str = path_parts[-3] 
            month_str = path_parts[-2]
            timestamp = filename.split('-')[2]
            day_str = timestamp[6:8]
        except Exception:
            print(f"Skipping malformed path: {key}")
            continue

        try:
            # Stream directly from S3
            response = s3_client.get_object(Bucket='commoncrawl', Key=key)
            # Fastwarc handles the stream
            stream = GZipStream(response['Body'])
            
            for record in ArchiveIterator(stream, record_types=WarcRecordType.response, func_filter=is_http):
                try:
                    uri = record.headers.get('WARC-Target-URI')
                    body_bytes = record.reader.read()
                    
                    # Encoding and Text Extraction
                    encoding = detect_encoding(body_bytes)
                    html = bytes_to_str(body_bytes, encoding)
                    text = extract_plain_text(html)
                    
                    # Metadata
                    http_date = record.http_date
                    http_last_modified = record.http_last_modified
                    http_charset = record.http_charset
                    surt_uri = surt(uri)
                    host = get_surt_host(uri)
                    
                    # Language Detection
                    r = detect_fast(text, n_results=3)
                    main_lang = r[0][0] if r else 'unknown'
                    langs = [x[0] for x in r]
                    confs = [float(x[1]) for x in r]

                    # Yield a dictionary representing the row
                    yield {
                        'uri': uri,
                        'text': text, # Storing full text
                        'html': html, # Optional: Storing full HTML
                        'main_lang': main_lang,
                        'langs': langs,
                        'confs': confs,
                        'http_date': http_date,
                        'http_last_modified': http_last_modified,
                        'http_charset': http_charset,
                        'surt_uri': surt_uri,
                        'host': host,
                        # Partition columns
                        'path': filename,
                        'year': year_str,
                        'month': month_str,
                        'day': day_str
                    }
                except Exception as e:
                    # Log internal record errors but don't stop the stream
                    continue
                    
        except Exception as e:
            print(f"Error processing WARC file {key}: {e}")

In [10]:
def main():
    
    # Initialize Spark with GCS connector support
    spark = SparkSession.builder \
        .appName("CommonCrawlProcessor") \
        .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar") \
        .config("spark.jars.packages", "com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.16") \
        .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
        .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
        .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
        .config("google.cloud.auth.service.account.json.keyfile", GCP_SA_KEY_PATH) \
        .getOrCreate()

    # A. Get the list of WARC paths (Driver side - lightweight)
    s3_client = boto3.client('s3', 
                             aws_access_key_id=AWS_ACCESS_KEY,
                             aws_secret_access_key=AWS_SECRET_KEY)
    
    paths_gz_key = 'crawl-data/CC-NEWS/2025/11/warc.paths.gz'
    response = s3_client.get_object(Bucket='commoncrawl', Key=paths_gz_key)
    decompressed_bytes = gzip.decompress(response['Body'].read())
    
    # Read paths into a list
    warc_paths = [line.decode('utf-8').strip() for line in decompressed_bytes.splitlines()]
    
    # B. Create a simple DataFrame of paths to distribute work
    # Repartition determines parallelism. e.g., if you have 1000 files and 100 partitions, 
    # each task processes ~10 files.
    paths_df = spark.createDataFrame([(p,) for p in warc_paths], ["warc_path"]).repartition(100)

    # C. Define Output Schema
    schema = StructType([
        StructField("uri", StringType(), True),
        StructField("text", StringType(), True),
        StructField("html", StringType(), True),
        StructField("main_lang", StringType(), True),
        StructField("langs", ArrayType(StringType()), True),
        StructField("confs", ArrayType(FloatType()), True),
        StructField("http_date", StringType(), True),
        StructField("http_last_modified", StringType(), True),
        StructField("http_charset", StringType(), True),
        StructField("surt_uri", StringType(), True),
        StructField("host", StringType(), True),
        # Partition cols
        StructField("path", StringType(), True),
        StructField("year", StringType(), True),
        StructField("month", StringType(), True),
        StructField("day", StringType(), True),
    ])

    # D. Execute Processing (Map Partitions)
    # mapPartitions is more efficient than map because we init the S3 client once per partition
    processed_rdd = paths_df.rdd.mapPartitions(process_warc_partition)
    
    # Convert back to DataFrame
    final_df = spark.createDataFrame(processed_rdd, schema=schema)

    # E. Write to GCS
    print(f"Writing to {GCS_BUCKET_ROOT}...")
    final_df.write \
        .mode("append") \
        .partitionBy("year", "month", "day", "path", "main_lang") \
        .parquet(GCS_BUCKET_ROOT)

    print("Job Complete.")
    spark.stop()

In [ ]:
main()

Writing to gs://gen-ai-tu/news/raw...


25/11/28 17:18:26 ERROR Executor: Exception in task 27.0 in stage 6.0 (TID 284)]
net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for datetime.timezone). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.api.python.SerDeUtil$.$anonfun$pythonToJava$2(SerDeUtil.scala:121)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.sca

Py4JJavaError: An error occurred while calling o157.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 27 in stage 6.0 failed 1 times, most recent failure: Lost task 27.0 in stage 6.0 (TID 284) (172.24.0.1 executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for datetime.timezone). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.api.python.SerDeUtil$.$anonfun$pythonToJava$2(SerDeUtil.scala:121)
	at scala.collecti

SerDeUtil.scala:121)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvaluatorFactory.scala:43)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:385)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFi

25/11/28 17:18:27 WARN TaskSetManager: Lost task 34.0 in stage 6.0 (TID 291) (172.24.0.1 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 27 in stage 6.0 failed 1 times, most recent failure: Lost task 27.0 in stage 6.0 (TID 284) (172.24.0.1 executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for datetime.timezone). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.api.python.SerDeUtil$.$anonfun$pythonToJava$2

In [15]:
from google.cloud import storage
import os

In [16]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )



In [17]:
os.listdir('../../../src/gen_ai_pipeline/gen_ai_pipeline/resources/')

['__pycache__', 'environment.sh', 'install_pixi_env.sh']

In [18]:
upload_blob("gen-ai-tu", '../../../src/gen_ai_pipeline/gen_ai_pipeline/resources/environment.sh', "artifacts/environment.sh")

File ../../../src/gen_ai_pipeline/gen_ai_pipeline/resources/environment.sh uploaded to artifacts/environment.sh.


In [13]:
upload_blob("gen-ai-tu", '../../../src/gen_ai_pipeline/gen_ai_pipeline/resources/install_pixi_env.sh', "artifacts/install_pixi_env.sh")

PreconditionFailed: 412 POST https://storage.googleapis.com/upload/storage/v1/b/gen-ai-tu/o?uploadType=multipart&ifGenerationMatch=0: {
  "error": {
    "code": 412,
    "message": "At least one of the pre-conditions you specified did not hold.",
    "errors": [
      {
        "message": "At least one of the pre-conditions you specified did not hold.",
        "domain": "global",
        "reason": "conditionNotMet",
        "locationType": "header",
        "location": "If-Match"
      }
    ]
  }
}
: ('Request failed with status code', 412, 'Expected one of', <HTTPStatus.OK: 200>)

In [16]:
upload_blob("gen-ai-tu", '../../../src/gen_ai_pipeline/gen_ai_pipeline/external/ccnews_extract_job.py', "artifacts/ccnews_extract_job.py")

File ../../../src/gen_ai_pipeline/gen_ai_pipeline/external/ccnews_extract_job.py uploaded to artifacts/ccnews_extract_job.py.


In [26]:
upload_blob("gen-ai-tu", '../../../src/gen_ai_pipeline/gen_ai_pipeline/external/ccnews_preprocess_job.py', "artifacts/ccnews_preprocess_job.py")

File ../../../src/gen_ai_pipeline/gen_ai_pipeline/external/ccnews_preprocess_job.py uploaded to artifacts/ccnews_preprocess_job.py.


In [25]:
upload_blob("gen-ai-tu", '../../../src/gen_ai_pipeline/gen_ai_pipeline/resources/install_pixi_preprocess.sh', "artifacts/install_pixi_preprocess.sh")

File ../../../src/gen_ai_pipeline/gen_ai_pipeline/resources/install_pixi_preprocess.sh uploaded to artifacts/install_pixi_preprocess.sh.
